In [ ]:
# Check Python Version
!python --version

In [ ]:
# Check Ubuntu Version
!lsb_release -a

In [ ]:
# Check CUDA/cuDNN Version
!nvcc -V && which nvcc

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import cudf
import gc
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import random
import torch
import torchvision
from torchvision import  models, transforms
from transformers import BertTokenizer, BertModel
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors
import torch.nn as nn
import torch.nn.functional as F
import os
import glob
from PIL import Image
import seaborn as sns
import cv2, matplotlib.pyplot as plt
import matplotlib.image as mpimg
from textwrap import wrap

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
path = '/content/drive/MyDrive/Shopee/'

In [ ]:
train_df = pd.read_csv(path+'train.csv')
# test_df = pd.read_csv(path+'test.csv')

In [ ]:
train_df.shape

In [ ]:
train_df.head()

In [ ]:
device = 'cuda'if torch.cuda.is_available() else 'cpu'
device

In [ ]:
tmp = train_df.groupby('label_group').posting_id.agg('unique').to_dict()
train_df['target'] = train_df.label_group.map(tmp)

In [ ]:
def show_random_img():
    # choose randomly two instances per each class
    labels_to_show = np.random.choice(train_df.label_group.unique(), 
                                      replace=False, size=24)
    img_to_show = []
    for label in labels_to_show:
        rows = train_df[train_df.label_group==label].copy()
        pair = np.random.choice([i for i in range(len(rows))], 
                                    replace=False, size=2)
        img_pair = rows.iloc[pair][['image', 'title']].values
        
        img_to_show += list(img_pair)
    
    fig, axes = plt.subplots(figsize = (18, 12), nrows=4,ncols=6)
    for imp, ax in zip(img_to_show, axes.ravel()):
        img = cv2.imread(path+'train_images/' + imp[0])
        title = '\n'.join(wrap(imp[1], 20))
        ax.set_title(title)
        ax.imshow(img)
        ax.axis('off')

    fig.tight_layout()

In [ ]:
show_random_img()

In [ ]:
class ResNetEmbedder(nn.Module):
    
    def __init__(self, device='cpu'):
        super(ResNetEmbedder, self).__init__()
        self.model = models.resnet50(pretrained=False)
        self.device = device
        path = '/content/drive/MyDrive/Shopee/resnet50-19c8e357.pth'
        self.model.load_state_dict(torch.load(path))
#         to freeze weights
        for param in self.model.parameters():
                param.requires_grad = False
        self.model.to(device)
        
    
    def transform(self, img):
        image_transform = torchvision.transforms.Compose(
            [
                torchvision.transforms.Resize(256),
                transforms.CenterCrop(224),
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize(
                    mean=(0.485, 0.456, 0.406), 
                    std=(0.229, 0.224, 0.225)
                ),
            ]
        )
        return image_transform(img)
    
    def forward(self, img):
        img_tr = self.transform(img).unsqueeze(0)
        img_tr = img_tr.to(self.device)
        features = self.model(img_tr).squeeze()
        return features

In [ ]:
model_img = ResNetEmbedder(device)

In [ ]:
def vectorize_img(img_path):
    img = Image.open(img_path).convert('RGB')
    model_img.eval()
    with torch.no_grad():
        output = model_img(img).cpu().numpy()
    return output

In [ ]:
%%time
train_df['resnet_v'] = train_df['image'].progress_apply(lambda x: vectorize_img(path+'train_images/' + x))

In [ ]:
# train_df.to_csv('/content/drive/MyDrive/Shopee/train_df_bkp.csv')
# train_df.to_pickle('/content/drive/MyDrive/Shopee/train_df_pkl')

In [ ]:
del model_img

In [ ]:
# normalize vectors
vectors = np.stack(train_df.resnet_v)
vectors = torch.Tensor(vectors).to(device)
vectors = F.normalize(vectors)

In [ ]:
# find the closest vectors for each image using cosine similarity via a threshold, based on ResNet embeddings

preds = []
CHUNK = 1024

CTS = len(train_df) // CHUNK

if len(train_df) % CHUNK != 0:
  CTS += 1

for j in range(CTS):
    
    a = j * CHUNK
    b = (j + 1) * CHUNK
    b = min(b, len(train_df))
    print('chunk', a, 'to', b)
    
    # COSINE SIMILARITY DISTANCE
    cts = torch.matmul( vectors, vectors[a:b].T).T
    cts = cts.cpu().numpy()
    
    for k in range(b-a):
        IDX = np.where(cts[k,] > 0.9)[0]
        o = train_df.iloc[IDX].posting_id.values
        preds.append(o)

del vectors, cts, IDX, o
_ = gc.collect()

In [ ]:
train_df['preds_resnet'] = preds
train_df.head()

In [ ]:
def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.target,row[col]) )
        return 2*n / (len(row.target)+len(row[col]))
    return f1score

In [ ]:
train_df['f1_resnet'] = train_df.apply(getMetric('preds_resnet'), axis=1)
print('CV score via image embeddings =', train_df.f1_resnet.mean())

In [ ]:
class BERTEmbedder(nn.Module):
    
    def __init__(self, device='cpu'):
        super(BERTEmbedder, self).__init__()
        self.bert_path = "../input/sentence-transformer/"
        self.model = BertModel.from_pretrained(self.bert_path)
#         to freeze weights
        for param in self.model.parameters():
                param.requires_grad = False
        self.model.to(device)
        
    def transform(self, txt):
        tokenizer = BertTokenizer.from_pretrained(self.bert_path)
        encoded_input  = tokenizer.encode_plus( txt, 
                                                truncation=True, 
                                                max_length=128,
                                                add_special_tokens=True,
                                                padding=True,
                                                return_tensors='pt').values()
        return encoded_input
    
    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask
        
    def forward(self, txt):
        inputs_ids, token_type_ids, attention_mask = self.transform(txt)
        inputs_ids, token_type_ids, attention_mask = inputs_ids.to(device), \
                                                token_type_ids.to(device), attention_mask.to(device)
        with torch.no_grad():
            encoded_layers = self.model(inputs_ids, 
                                        attention_mask=attention_mask, 
                                        token_type_ids=token_type_ids)
        features = self.mean_pooling(encoded_layers, attention_mask)
        return features

In [ ]:
model_txt = BERTEmbedder(device)

In [ ]:
def vectorize_txt(txt):
    model_txt.eval()
    with torch.no_grad():
        output = model_txt(txt).cpu().numpy()
    return output

In [ ]:
%%time
train_df['sbert_v'] = train_df['title'].progress_apply(lambda x: vectorize_txt(x))

In [ ]:
del model_txt

In [ ]:
vectors = np.stack(train_df.sbert_v).squeeze(1)
vectors = torch.Tensor(vectors).to(device)
vectors = F.normalize(vectors)

In [ ]:
preds = []
CHUNK = 1024

print('Finding similar titles...')
CTS = len(train_df)//CHUNK
if len(train_df)%CHUNK!=0: CTS += 1
for j in range( CTS ):
    
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b,len(train_df))
    print('chunk',a,'to',b)
    
    # COSINE SIMILARITY DISTANCE
    cts = torch.matmul( vectors, vectors[a:b].T).T
    cts = cts.cpu().numpy()
    
    for k in range(b-a):
        IDX = np.where(cts[k,]>0.95)[0]
        o = train_df.iloc[IDX].posting_id.values
        preds.append(o)

del vectors, cts, IDX, o
_ = gc.collect()

In [ ]:
train_df['preds_sbert'] = preds
train_df.head()

In [ ]:
del preds

In [ ]:
train_df['f1_sbert'] = train_df.apply(getMetric('preds_sbert'), axis=1)
print('CV score via title embeddings =', train_df.f1_sbert.mean())

In [ ]:
def concat():
    def cat(row):
        comm = np.concatenate([row.resnet_v,row.sbert_v.squeeze()])
        return comm
    return cat

In [ ]:
train_df['concat_v'] = train_df.progress_apply(concat(), axis=1)

In [ ]:
vectors = np.stack(train_df.concat_v)

In [ ]:
KNN = 50
model = NearestNeighbors(n_neighbors=KNN)
model.fit(vectors)

In [ ]:
preds = []
CHUNK = 1024*4

print('Finding similar images...')
CTS = len(vectors)//CHUNK
if len(vectors)%CHUNK!=0: CTS += 1
for j in range( CTS ):
    
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b,len(vectors))
    print('chunk',a,'to',b)
    distances, indices = model.kneighbors(vectors[a:b,])
    
    for k in range(b-a):
        IDX = np.where(distances[k,]<35.0)[0]
        IDS = indices[k,IDX]
        o = train_df.iloc[IDS].posting_id.values
        preds.append(o)
        
del model, distances, indices, vectors, IDX, o, IDS
_ = gc.collect()

In [ ]:
train_df['preds_concat'] = preds
train_df.head()

In [ ]:
del preds

In [ ]:
train_df['f1_concat'] = train_df.apply(getMetric('preds_concat'), axis=1)
print('CV score for baseline =', train_df.f1_concat.mean())

In [ ]:
tmp = train_df.groupby('image_phash').posting_id.agg('unique').to_dict()
train_df['preds_phash'] = train_df.image_phash.map(tmp)
train_df.head()

In [ ]:
del tmp

In [ ]:
dataset_gf = cudf.DataFrame(train_df[['posting_id', 'title']])

In [ ]:
model = TfidfVectorizer(stop_words='english', binary=True, max_features=25_000)
text_embeddings = model.fit_transform(dataset_gf.title)

In [ ]:
del model

In [ ]:
preds = []
CHUNK = 1024

print('Finding similar titles...')
CTS = len(train_df)//CHUNK
if len(train_df)%CHUNK!=0: CTS += 1
for j in range( CTS ):
    
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b,len(train_df))
    print('chunk',a,'to',b)
    
    # COSINE SIMILARITY DISTANCE
    cts = text_embeddings.dot(text_embeddings[a:b].T).T.toarray()
    
    for k in range(b-a):
        IDX = cupy.where(cts[k,]>0.7)[0]
        o = train_df.iloc[cupy.asnumpy(IDX)].posting_id.values
        preds.append(o)
        
del text_embeddings, IDX, o, cts
_ = gc.collect()

In [ ]:
train_df['preds_tfidf'] = preds

In [ ]:
del preds

In [ ]:
train_df['f1_tfidf'] = train_df.apply(getMetric('preds_tfidf'), axis=1)
print('CV score for baseline =', train_df.f1_tfidf.mean())

In [ ]:
def combine_for_sub(row):
    x = np.concatenate([row.preds_concat,row.preds_phash, row.preds_tfidf])
    return ' '.join( np.unique(x) )

def combine_for_train(row):
    x = np.concatenate([row.preds_concat,row.preds_phash, row.preds_tfidf])
    return list(np.unique(x))

In [ ]:
train_df['matches'] = train_df.apply(combine_for_train, axis=1)

In [ ]:
train_df.to_pickle('train_data.pkl')

In [ ]:
train_df[['posting_id', 'matches']].to_csv('submission.csv',index=False)

In [ ]:
subm = pd.read_csv('submission.csv')
subm.head()

In [ ]:
train_df['f1_final'] = train_df.apply(getMetric('matches'), axis=1)
print('CV score for baseline =', train_df.f1_final.mean())